In [2]:
from pymongo import MongoClient
import pymongo

In [2]:
client = MongoClient()

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

# 导入测试数据

# pymongo 使用

In [8]:
# 连接

In [12]:
client = MongoClient("mongodb://localhost:27017")
client = MongoClient()

In [13]:
db = client.test
db = client['test']

In [6]:
client

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'testsddfgfjss')

# 获取集合

In [14]:
db.restaurants

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'restaurants')

# 插入数据

## 插入一个Document

In [16]:
from datetime import datetime
result = db.restaurants.insert_one(
    {
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Manhattan",
        "cuisine": "Italian",
        "grades": [
            {
                "date": datetime.strptime("2014-10-01", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("2014-01-16", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "Vella",
        "restaurant_id": "41704620"
    }
)

In [17]:
##  插入的ID
result

ObjectId('589c2e0cd3ac6b1a2515b175')

## 插入多个Document

In [15]:
db.restaurants.count()

25360

In [200]:
result = db.restaurants.insert_many([{'x': i,'y':34} for i in range(3)])

In [23]:
data = [
    {},
    {
        'name1':1,
        '2':2
    },
    {
        'hello':'test'
    }
    
]

In [24]:
result = db.restaurants.insert_many(data)

In [25]:
result.inserted_ids

[ObjectId('589c2f0dd3ac6b1a2515b17c'),
 ObjectId('589c2f0dd3ac6b1a2515b17d'),
 ObjectId('589c2f0dd3ac6b1a2515b17e')]

In [26]:
for x in result:
    print(x)

TypeError: 'InsertManyResult' object is not iterable

# 查找 查询 数据

##  查询所有数据

In [57]:
cursor.count()

25367

In [56]:
cursor[56]

IndexError: no such item for Cursor instance

In [43]:
cursor = db.restaurants.find()

### 基于位置索引查询
cursor[0]

## 相等条件查询 Specify Equality Conditions¶

### 查询顶级字段数据

In [58]:
cursor = db.restaurants.find({"borough": "Manhattan"})


In [59]:
cursor.count()

10261

### 通过内嵌字段查询 Query by a Field in an Embedded Document，

In [229]:
cursor = db.restaurants.find({"address.zipcode": "10075"})

In [230]:
cursor[2]

{'_id': ObjectId('589bd9f0d1dc797515fdb166'),
 'address': {'building': '1133',
  'coord': [-73.958885, 40.7745559],
  'street': 'Lexington Avenue',
  'zipcode': '10075'},
 'borough': 'Manhattan',
 'cuisine': 'Italian',
 'grades': [{'date': datetime.datetime(2014, 8, 11, 0, 0),
   'grade': 'A',
   'score': 11},
  {'date': datetime.datetime(2013, 12, 10, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2013, 6, 10, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 6, 8, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2012, 1, 25, 0, 0), 'grade': 'A', 'score': 8},
  {'date': datetime.datetime(2011, 9, 13, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Don Filippo Restaurant',
 'restaurant_id': '40372417'}

### 通过数组内字段查询 Query by a Field in an Array

In [231]:
cursor = db.restaurants.find({"grades.grade": "B"})

## 应用操作符查询 Specify Conditions with Operators

### > 大于  Greater Than Operator ($gt)

In [232]:
cursor = db.restaurants.find({"grades.score": {"$gt": 30}})

### < 小于操作 Less Than Operator ($lt)

In [233]:
cursor = db.restaurants.find({"grades.score": {"$lt": 10}})

$gte，>=; $lte <=; 
其他操作符 ，详见 https://docs.mongodb.com/manual/reference/operator/query/

##  组合查询  Combine Conditions

### Logical AND
#直接添加 条件

In [136]:
cursor = db.restaurants.find({"cuisine": "Italian", "grades.score": {"$lt": 10}})

In [137]:
cursor = db.restaurants.find({"cuisine": "Italian", "address.zipcode": "10075"})

In [234]:
cursor = db.restaurants.find({"$and": [{"cuisine": "Italian"}, {"address.zipcode":"10075"}]})

### Logical OR

In [144]:
cursor = db.restaurants.find(
    {"$or": [{"cuisine": "Italian"}, {"address.zipcode": "10075"}]})

## 排序 Sort Query Results

In [148]:
cursor = db.restaurants.find().sort([
    ("borough", pymongo.ASCENDING),
    ("address.zipcode", pymongo.ASCENDING)
])

# Update Data with PyMongo

## 更新特定键 Update Specific Fields

In [153]:
'''
updates the first document with name equal to "Juni", 
using the $set operator to update the cuisine field and 
the $currentDate operator to update the lastModified field with the current date.
'''

result = db.restaurants.update_one(
    {"name": "Juni"},
    {
        "$set": {
            "cuisine": "American (New)"
        },
        "$currentDate": {"lastModified": True}
    }
)

In [ ]:
返回一个UpdateResult， 具有本次更新的相关数据，如result.modified_count

### 更新内嵌的键值对 Update an Embedded Field

In [160]:
#To update a field within an embedded document, use the dot notation
result = db.restaurants.update_one(
    {"restaurant_id": "41156888"},
    {"$set": {"address.street": "East 31st Street"}}
)

In [161]:
## Update Multiple Documents
'''
The following operation updates all documents that have address.
zipcode field equal to "10016" and cuisine field equal to "Other", 
setting the cuisine field to "Category To Be Determined" 
and the lastModified field to the current date.
'''
result = db.restaurants.update_many(
    {"address.zipcode": "10016", "cuisine": "Other"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": True}
    }
)

In [162]:
## Replace a Document

In [174]:
result = db.restaurants.replace_one(
    {"restaurant_id": "41704620"},
    {
        "name": "Vella 2",
        "address": {
            "coord": [-73.9557413, 40.7720266],
            "building": "1480",
            "street": "2 Avenue",
            "zipcode": "10075"
        }
    }
)

# 删除数据 Remove Data

##  删除所有符合条件数据 Remove All Documents That Match a Condition

In [178]:
result = db.restaurants.delete_many({"borough": "Manhattan"})

## 删除所有数据 Remove All Documents

In [ ]:
result = db.restaurants.delete_many({})

## 删除整个集合 Drop a Collection

In [165]:
db.restaurants.drop()

# 数据聚合 Data Aggregation

## Group Documents by a Field and Calculate Count

In [62]:
cursor = db.restaurants.aggregate(
    [
        {"$group": {"_id": "$borough", "count": {"$sum": 1}}}
    ]
)

In [63]:
for x in cursor:
    print(x)

{'_id': None, 'count': 12}
{'_id': 'Missing', 'count': 102}
{'_id': 'Staten Island', 'count': 1938}
{'_id': 'Brooklyn', 'count': 12172}
{'_id': 'Bronx', 'count': 4676}
{'_id': 'Queens', 'count': 11312}
{'_id': 'Manhattan', 'count': 20522}


## Filter and Group Documents

In [169]:
cursor = db.restaurants.aggregate(
    [
        {"$match": {"borough": "Queens", "cuisine": "Brazilian"}},
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}}
    ]
)


# Indexes with PyMongo

In [176]:
db.restaurants.create_index([
    ("cuisine", pymongo.ASCENDING),
    ("address.zipcode", pymongo.DESCENDING)
])

'cuisine_1_address.zipcode_-1'

# 参考：  https://docs.mongodb.com/getting-started/python/